In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, expr
from pyspark.sql.types import StructType, StructField, StringType, LongType, DoubleType, IntegerType, ArrayType, DateType
import sys
import os
from delta import DeltaTable
from delta.tables import *


from pyspark.sql import DataFrame
from pyspark.sql.utils import AnalysisException
import io
import json

In [0]:
def create_spark_session():
    return SparkSession \
        .builder \
        .appName("File Streaming Demo") \
        .master("local[3]") \
        .config("spark.databricks.delta.retentionDurationCheck.enabled", 'false')\
        .config("spark.databricks.delta.schema.autoMerge.enabled", "true")\
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
        .enableHiveSupport()\
        .getOrCreate()

In [0]:
spark = create_spark_session()

### History em Delta Tables <br>
atravez do método **DeltaTable.forPath(spark, caminho_Delta_table)**

**Importante** Uma DeltaTable não é um spark dataframe, então caso seja necessário utilizar operações e ações de um <br> 
spark dataframe existem duas formas :
* Carregar os dados gravados no formato delta para um dataframe pyspark,utilizar o spark.read.format('delta').load(caminho_Delta_table) 

* Carregar uma DeltaTable com **DeltaTable.forPath(spark, caminho_Delta_table)** e usar o metodo **toDf()** , que retorna um dataframe spark de uma tabela delta

In [0]:
location_tb = '/FileStore/bronze/aula_delta/products'
dbutils.fs.rm(location_tb,True)

In [0]:
schema = StructType([ 
    StructField("product_id",IntegerType(),True), \
    StructField("product_name",StringType(),True),
    StructField("product_category_name",StringType(),True), \
    StructField("product_price", DoubleType(), True),
    StructField("status",StringType(),True),
  ])

data_product = [(1,'perfurme magico','perfumaria',100.5,'Inative'),
                (2,'replica vaso ming importado','artes',70.75,'Active'),
                (3,'raquete de tenis nacional','esporte_lazer',365.00,'Active'),
                (4,'mordedor','bebes',27.25,'Inative'),
                (5,'televisor 46 polegadas 4k','utilidades_domesticas',2555.55,'Active'),
                (6,'PS','utilidades_domesticas',2555.55,'Active')
               ]


df_products = spark.createDataFrame(data=data_product,schema=schema)


In [0]:

location_tb = '/FileStore/bronze/aula_delta/products'
merge_condition = "tgt.product_id = src.product_id"

columns = [
    StructField('product_id', IntegerType(), True),
    StructField('product_name', StringType(), True),
    StructField('product_category_name', StringType(), True),
    StructField('product_price', DoubleType(), True),
    StructField('status', StringType(), True)
]

if (DeltaTable.isDeltaTable(spark, location_tb)):
    print('tabela delta existente')
    deltaTable = DeltaTable.forPath(spark, location_tb)
    deltaTable.alias('tgt') \
        .merge(
            df_products.alias('src'),
            merge_condition
        ) \
        .whenMatchedUpdateAll() \
        .whenNotMatchedInsertAll() \
        .execute()
else:
    print('tabela delta inexistente')    
    DeltaTable \
            .create(spark) \
            .addColumns(columns) \
            .location(location_tb) \
            .execute()
    deltaTable = DeltaTable.forPath(spark, location_tb)
    deltaTable.alias('tgt') \
        .merge(
            df_products.alias('src'),
            merge_condition
        ) \
        .whenMatchedUpdateAll() \
        .whenNotMatchedInsertAll() \
        .execute()


In [0]:
from delta import DeltaTable
from delta.tables import *

In [0]:

location_tb = '/FileStore/bronze/aula_delta/products'
deltaTable = DeltaTable.forPath(spark, location_tb)
fullHistoryDF = deltaTable.history()
fullHistoryDF.display()

In [0]:
fullHistoryDF.select("version", "timestamp", "operation", "operationParameters").display(10, False)


### Descrição dos campos retornados pelo history ###

Column|Type|Description
------|----|-----------
version|	long|	Table version generated by the operation.
timestamp|	timestamp|	When this version was committed.
userId|	string|	ID of the user that ran the operation.
userName|	string|	Name of the user that ran the operation.
operation|	string|	Name of the operation.
operationParameters|	map|	Parameters of the operation (for example, predicates.)
job|	struct|	Details of the job that ran the operation.
notebook|	struct|	Details of notebook from which the operation was run.
clusterId|	string|	ID of the cluster on which the operation ran.
readVersion|	long|	Version of the table that was read to perform the write operation.
isolationLevel|	string|	Isolation level used for this operation.
isBlindAppend|	boolean|	Whether this operation appended data.
operationMetrics|	map|	Metrics of the operation (for example, number of rows and files modified.)
userMetadata|	string|	User-defined commit metadata if it was specified

### Versioning em Delta Tables <br>
É possivel carregar um dataframe spark com uma versão especifica existente, somente adicionando a option <br>
**option("versionAsOf", numero_da_versao)** ,  no exemplo, basta substituir o numero_da_versão por qualquer número de versão existente.

In [0]:
location_tb = '/FileStore/bronze/aula_delta/products'
df_products_v0 = spark.read.format('delta').option("versionAsOf", 0).load(location_tb)

In [0]:
df_products_v0.display()

In [0]:
location_tb = '/FileStore/bronze/aula_delta/products'
df_products_v1 = spark.read.format('delta').option("versionAsOf", 1).load(location_tb)

In [0]:
df_products_v1.display()

In [0]:
spark.read.format("delta").option("versionAsOf", "1").load(location_tb).display()

### Restore de versions em Delta Tables <br>
É possivel carregar um dataframe spark com uma versão especifica existente, somente adicionando a option <br>
**option("versionAsOf", numero_da_versao)** ,  no exemplo, basta substituir o numero_da_versão por qualquer número de versão existente.

In [0]:
location_tb = '/FileStore/bronze/aula_delta/products'
deltaTable = DeltaTable.forPath(spark, location_tb)


No exemplo abaixo, utilizamos a versão 0, onde somente criamos a delta table sem dados, o comando show retorna somente o cabeçalho dos dados como esperado <br>
**deltaTable.restoreToVersion(0)**

In [0]:
deltaTable.restoreToVersion(0)

In [0]:
df_from_delta = deltaTable.toDF()
df_from_delta.display()

Para demonstrar o a mudança dos dados, escolhemos a versão 1, onde o comando de **MERGE** foi realizado,  com isso uma nova <br> versão é criada com os dados da versão 1 , a versão criada é sempre mais atual. <br>
 **deltaTable.restoreToVersion(1)**

In [0]:
deltaTable.restoreToVersion(1)

In [0]:
df_from_delta = deltaTable.toDF()
df_from_delta.display()

In [0]:
fullHistoryDF = deltaTable.history()
fullHistoryDF.display()

### Vaccum em Delta Tables <br>
É possivel remover dados antigos de uma Delta Table utilizando o comando **Vacuum** . <br>
Essa opção se torna interessante por uma Delta Table vai acumular diversas versão criando maior custo de armazenagem de dados <br>
Após rodar o comando **vacuum** , os dados serão permanentemente removidos da Delta Table, o comando **vacuum** deve ser usado com cuidado <br>
e combinado com uma estrátégia de retenção de dados. <br>
No Exemplo abaixo o comando **vacuum** é disparado com o periodo default de retenção de dados. <br>
No segundo exemplo, o comando **vacuum** é disparado removendo versão com mais de 10 horas.Como o periodo de retenção é pequeno,<br>
o spark não vai executar o comando e vai informar que é necessário acrescentar a configuração
**spark.databricks.delta.retentionDurationCheck.enabled = false** no Spark Session

In [0]:

location_tb = '/FileStore/bronze/aula_delta/products'
deltaTable = DeltaTable.forPath(spark, location_tb)
fullHistoryDF = deltaTable.history()
fullHistoryDF.display()

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
3,2024-02-13T14:03:23.000+0000,523858428647384,jaderablima@gmail.com,RESTORE,"Map(version -> 1, timestamp -> null)",null,List(1220161194661759),0213-101706-n0ahm20b,2,Serializable,false,"Map(numRestoredFiles -> 6, removedFilesSize -> 0, numRemovedFiles -> 0, restoredFilesSize -> 10949, numOfFilesAfterRestore -> 6, tableSizeAfterRestore -> 10949)",null,Databricks-Runtime/12.2.x-scala2.12
2,2024-02-13T14:02:22.000+0000,523858428647384,jaderablima@gmail.com,RESTORE,"Map(version -> 0, timestamp -> null)",null,List(1220161194661759),0213-101706-n0ahm20b,1,Serializable,false,"Map(numRestoredFiles -> 0, removedFilesSize -> 10949, numRemovedFiles -> 6, restoredFilesSize -> 0, numOfFilesAfterRestore -> 0, tableSizeAfterRestore -> 0)",null,Databricks-Runtime/12.2.x-scala2.12
1,2024-02-13T13:53:44.000+0000,523858428647384,jaderablima@gmail.com,MERGE,"Map(predicate -> [""(product_id#26577 = product_id#26142)""], matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""actionType"":""insert""}], notMatchedBySourcePredicates -> [])",null,List(1220161194661759),0213-101706-n0ahm20b,0,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 6, numTargetBytesAdded -> 10949, numTargetBytesRemoved -> 0, numTargetDeletionVectorsAdded -> 0, numTargetRowsMatchedUpdated -> 0, executionTimeMs -> 2733, numTargetRowsInserted -> 6, numTargetRowsMatchedDeleted -> 0, scanTimeMs -> 893, numTargetRowsUpdated -> 0, numOutputRows -> 6, numTargetDeletionVectorsRemoved -> 0, numTargetRowsNotMatchedBySourceUpdated -> 0, numTargetChangeFilesAdded -> 0, numSourceRows -> 6, numTargetFilesRemoved -> 0, numTargetRowsNotMatchedBySourceDeleted -> 0, rewriteTimeMs -> 1152)",null,Databricks-Runtime/12.2.x-scala2.12
0,2024-02-13T13:53:38.000+0000,523858428647384,jaderablima@gmail.com,CREATE TABLE,"Map(isManaged -> false, description -> null, partitionBy -> [], properties -> {})",null,List(1220161194661759),0213-101706-n0ahm20b,null,WriteSerializable,true,Map(),null,Databricks-Runtime/12.2.x-scala2.12


In [0]:
conf = spark.sparkContext.getConf().get('spark.databricks.delta.retentionDurationCheck.enabled')
print(conf)

None


In [0]:
deltaTable.vacuum()        # vacuum files not required by versions older than the default retention period



In [0]:
fullHistoryDF = deltaTable.history()
fullHistoryDF.display()

In [0]:
deltaTable.vacuum(169) 